In [22]:
import torch as th
import numpy as np
import torchvision
import cv2
import math

In [2]:
th.cuda.is_available()

True

In [3]:
torchvision.__version__

'0.16.2+cu121'

In [4]:
dev = th.device(0)

In [5]:
dev

device(type='cuda', index=0)

In [6]:
import torch
import torchvision

device = 'cuda'
boxes = torch.tensor([[0., 1., 2., 3.]]).to(device)
scores = torch.randn(1).to(device)
iou_thresholds = 0.5

print(torchvision.ops.nms(boxes, scores, iou_thresholds))

tensor([0], device='cuda:0')


In [7]:
yolov5s = th.hub.load("ultralytics/yolov5","yolov5s",pretrained=True)

Using cache found in C:\Users\pc/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-12-30 Python-3.11.5 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2060 SUPER, 8192MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [8]:
next(yolov5s.parameters()).is_cuda

True

In [9]:
yolov5s.cuda()

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [10]:
next(yolov5s.parameters()).is_cuda

True

In [11]:
idx_to_class ={0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mouse',
 65: 'remote',
 66: 'keyboard',
 67: 'cell phone',
 68: 'microwave',
 69: 'oven',
 70: 'toaster',
 71: 'sink',
 72: 'refrigerator',
 73: 'book',
 74: 'clock',
 75: 'vase',
 76: 'scissors',
 77: 'teddy bear',
 78: 'hair drier',
 79: 'toothbrush'}

In [24]:
def convert_xy_to_centerpts(x1,y1,x2,y2):
    return (x1+x2)/2,(y1+y2)/2

In [33]:
class Tracker:
    def __init__(self):
        self.tracking_points = dict()
        self.ID_NUM = 0

    def update(self,detections):
        new_tracked_items = dict()
        for detection in detections:
            cx,cy = convert_xy_to_centerpts(*detection)
            is_matched = False
            for id,tp in self.tracking_points.items():
                ocx,ocy = convert_xy_to_centerpts(*tp)
                dst = math.hypot(cx-ocx,cy-ocy)
                if dst <= 30:
                    new_tracked_items[id] = detection
                    self.tracking_points.pop(id)
                    is_matched = True
                    break

            if is_matched == False:
                new_tracked_items[self.ID_NUM] = detection
                self.ID_NUM += 1

        self.tracking_points = new_tracked_items.copy()

In [12]:
VIDEO_LOC = r"E:\PyImage_ComputerVision\ObjectDetectionYolo\WorkingData\InputData\TrafficVideo1.mp4"

In [13]:
video1 = cv2.VideoCapture(VIDEO_LOC)

In [14]:
video1.isOpened()

True

In [16]:
FPS = video1.get(cv2.CAP_PROP_FPS)
FW = video1.get(cv2.CAP_PROP_FRAME_WIDTH)
FH = video1.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [18]:
FPS,FW,FH

(29.97, 1920.0, 1080.0)

In [20]:
video2 = cv2.VideoWriter("E:\PyImage_ComputerVision\ObjectDetectionYolo\WorkingData\OutputData\TF1.avi",cv2.VideoWriter_fourcc('M','P','4','S'),FPS,[int(FW),int(FH)])

In [36]:
video1 = cv2.VideoCapture(VIDEO_LOC)
FPS = video1.get(cv2.CAP_PROP_FPS)
FW = video1.get(cv2.CAP_PROP_FRAME_WIDTH)
FH = video1.get(cv2.CAP_PROP_FRAME_HEIGHT)

video2 = cv2.VideoWriter("E:\PyImage_ComputerVision\ObjectDetectionYolo\WorkingData\OutputData\TF1.avi",cv2.VideoWriter_fourcc('M','P','4','S'),FPS,[int(FW),int(FH)])
tracker = Tracker()

while video1.isOpened():
    ret,frame = video1.read()
    yolov5s
    if ret == False:
        break
    else:
        preds = yolov5s(frame).pred[0]
        detections = []
        for pred in preds:
            cls = int(pred[5])
            if cls in [2,5,7]:
                x1 = pred[0]
                y1 = pred[1]
                x2 = pred[2]
                y2 = pred[3]
                conf = float(pred[4])
                detections.append((x1,y1,x2,y2))
        tracker.update(detections)
        for id,bbox in tracker.tracking_points.items():
            x1,y1,x2,y2 = bbox
            cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),(0,0,0),2)
            cx,cy = convert_xy_to_centerpts(x1,y1,x2,y2)
            cv2.circle(frame,(int(cx),int(cy)),2,(0,255,0),-1)
            cv2.putText(frame,f"{id}",(int(x1),int(y1)),cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,255))
        video2.write(frame)
video2.release()
video1.release()

In [25]:
fg = (2,2,4)

In [28]:
fg1 = (*fg,3)

In [29]:
fg1

(2, 2, 4, 3)